# Краулеры

**План**

1. Что такое краулеры?
2. Как написать простой краулер?
3. Блокировки и способы их обхода

## Что такое краулеры?

Краулеры - это боты/рпограммы, которые "ползают" по страницам сайта и собирают информацию. Все чаще использование таких программ запрещается правилами пользования сайтами, поэтому это формально нехорошо. Но так продолжают делать и это надо уметь. Запрещают по 2 основным причинам: не хотят делиться данными и боятся, что вы уроните сервер (если сайт маленький и сервер не очень, то это довольно легко). Поэтому нужно собирать данные аккуратно, чтобы вас а) не заблокировали по IP и б) вы не навредили серверу

## Как написать простой краулер?

In [ ]:
import requests
from pprint import pprint

Вместо отдельных запросов лучше создать сессию, которая позволит хранить информацию между запросами и поддерживать то же соединение и не создавать каждый раз все заново, что влияет на производительность.

In [80]:
session = requests.session()

Попробуем сделать запрос, просто вместо requests.get мы пишет session.get.

In [3]:
response = session.get('https://ru.wikipedia.org')

Может найти значения нашего IP-адреса

In [2]:
response.headers['X-Client-IP']

'188.255.82.7'

Посмотреть на headers запроса

In [4]:
pprint(dict(response.headers))

{'Accept-Ranges': 'bytes',
 'Age': '2809',
 'Cache-Control': 'private, s-maxage=0, max-age=0, must-revalidate',
 'Connection': 'keep-alive',
 'Content-Encoding': 'gzip',
 'Content-Language': 'ru',
 'Content-Length': '27633',
 'Content-Type': 'text/html; charset=UTF-8',
 'Date': 'Thu, 18 Nov 2021 10:01:03 GMT',
 'Last-Modified': 'Thu, 18 Nov 2021 10:01:00 GMT',
 'NEL': '{ "report_to": "wm_nel", "max_age": 86400, "failure_fraction": 0.05, '
        '"success_fraction": 0.0}',
 'P3p': 'CP="See https://ru.wikipedia.org/wiki/Special:CentralAutoLogin/P3P '
        'for more info."',
 'Permissions-Policy': 'interest-cohort=()',
 'Report-To': '{ "group": "wm_nel", "max_age": 86400, "endpoints": [{ "url": '
              '"https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" '
              '}] }',
 'Server': 'mw1395.eqiad.wmnet',
 'Server-Timing': 'cache;desc="hit-front", host;desc="cp3060"',
 'Strict-Transport-S

### Стратегии сбора данных


По сути краулеры выполняют сбор страниц (их html) как мы это делали на прошлом занятии, но делают они это циклами (или циклами циклов). Можно выделить разные стратегии сбора данных:
    
**По типу навигации**

1. Все страницы со ссылками имеют удобные номера ("https://ficbook.net/fanfiction/no_fandom/originals?p=2"), обычно просто p=(число) или page=(число). В этом случае вам нужно просто подставлять цифры
2. Страницы называются как-то не структурированно (например, по названиям блоков). Тут нужно собирать ссылки на эти страницы и потом по ним ходить и собирать конечные странички.

**По скорости обновления**

1. Если сайт довольно статичный по контенту (медленно появляются и удаляются материалы), то можно сначал собрать ссылки, а потом по ним ходить
2. Если сайт очень динамичный по контенту (например, объявления на крупном сайте), вам нужно при получении страничкии ссылок сразу их обходить, а потом переходить к следующей, потому что ко времени получения исчерпывающего списка ссылок по сайту многие будут уже удалены или недоступны



## Блокировки и способы их обхода

Для того, чтобы предотвратить автоматический сбор информации с некого сайта, применяются различные инструменты, которые определяют роботов и блокируют запросы с адресов, которые были классифицированы как роботы. Чтобы не заблокировали домашний/учебный ip, лучше сразу задуматься об этих мерах и предотвратить возможные проблемы. Кстати, Википедия не блокирует и можно спокойно скачивать без каких-либо проблем.

Чтобы их обойти, можно попробовать несколько инструментов:
1. time.sleep(x) - задержка между запросами, чтобы слишком большая скорость запросов не показалась подозрительной или ваши запросы не уронили сервер небольшого ресурса (например, региональной газеты)
2. time.sleep(случайный промежуток времени) - это более хитрая версия, когда время задержки - это случайное число из некоторого отрезка (модуль random)
3. изобразить браузер - при запросе отправляется информация о том, из какого приложения пришел запрос (например, Googlr Chrome), запросы сделанные из браузера больше похожи на человеческие, для этого нужно задать user-agent в параметрах (а его выбирать случайно с помощью fake_useragent)
4. использовать прокси - существуют ресурсы с бесплатными списками открытых прокси, через которые можно пропускать ваш запрос и сервер будет думать, что запросы приходят из разных мест (anonymous и elite классы прокси)

### Пауза между запросами

In [8]:
import time
from datetime import datetime

In [9]:
for _ in range(5):
    response = session.get('https://ru.wikipedia.org')
    print(datetime.now())
    time.sleep(1)

2021-11-18 13:49:57.752388
2021-11-18 13:49:58.897628
2021-11-18 13:50:00.025211
2021-11-18 13:50:01.160610
2021-11-18 13:50:02.301471


### Притвориться нормальным браузером

In [12]:
from fake_useragent import UserAgent

Можно настроить так, чтобы не проверять безопасность соединения, что иногда вызывает ошибки. Но это можно делать с сайтмаи, которым вы доверяете.

In [15]:
ua = UserAgent(verify_ssl=False)

In [16]:
headers = {'User-Agent': ua.random}
print(headers)
response = session.get('https://ru.wikipedia.org', headers=headers)

{'User-Agent': 'Mozilla/5.0 (X11; OpenBSD i386) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36'}


### Пауза между запросами (случайное время)

In [19]:
import random

random.uniform позволяет получить случайное число из отрезка

In [21]:
random.uniform(1, 3)

1.3048657080924062

In [20]:
for _ in range(5):
    response = session.get('https://ru.wikipedia.org')
    print(datetime.now())
    time.sleep(random.uniform(1.1, 5.2))

2021-11-18 13:53:40.490130
2021-11-18 13:53:45.131865
2021-11-18 13:53:47.951820
2021-11-18 13:53:51.541653
2021-11-18 13:53:56.309881


### Подключение через прокси

Прокси-сервер — это дополнительное звено между вами и интернетом, через него пойдет подключение и сайт не будет знать, что это вы посылаете запрос.

Адреса прокси можно взять со специальных сайтов, например, [https://hideip.me/ru/proxy/httplist](https://hideip.me/ru/proxy/httplist). И потом проверить, что они рабочие, прежде чем использовать [https://checkerproxy.net/](https://checkerproxy.net/)

In [35]:
known_proxy_ip = '20.119.120.226:59394'
proxy = {'http': known_proxy_ip, 'https': known_proxy_ip}
response = session.get('https://ru.wikipedia.org', proxies=proxy)
print(response.headers['X-Client-IP'])

20.55.91.13


## Примеры

### Пример 1

Давайте обкачаем немного новостей с сайта вышки.

1. Страницы имеют вид "https://www.hse.ru/news/page1.html", поэтому можно просто идти циклом.
2. Достанем дату публикации, заголовок, краткое описание (из станицы со списком новостей), текст полной статьи и метки (из самой страницы новости)
3. Положим в базу

In [36]:
import sqlite3
from html import unescape
from bs4 import BeautifulSoup
import re

In [97]:
conn = sqlite3.connect('hse_news.db')
cur = conn.cursor()

Создаем базу данных, где будем хранить информацию

In [98]:
cur.execute("""
CREATE TABLE IF NOT EXISTS texts 
(id INTEGER PRIMARY KEY AUTOINCREMENT, hse_id text, pub_year int, pub_month int, 
pub_day int, title text, short_text text, full_text text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS tags 
(id int PRIMARY KEY, tag_name text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS text_to_tag 
(id INTEGER PRIMARY KEY AUTOINCREMENT, id_text int, id_tag int) 
""")

conn.commit()
conn.close()

### Отработаем процесс на одной странице

**Шаг 1. Найти страницы**

Посмотрим, как устроены новости и скачаем одну страницу

In [39]:
page_number = 1
url = f'https://www.hse.ru/news/page{page_number}.html'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

Распарсим с помощью BeautifulSoup

In [40]:
soup = BeautifulSoup(page, 'html.parser')

Найдем отдельные посты

In [44]:
news = soup.find_all('div', {'class': 'post'})

In [45]:
len(news)

10

Найдем заголовок-ссылку и запомним текст заголовка

In [47]:
title_obj = news[0].find('a')
title_obj

<a class="link link_dark2 no-visited" href="/news/science/530853899.html">Экономисты объяснили, почему продавцам на аукционе выгодно держать свою цену в тайне</a>

In [48]:
title = title_obj.text
title

'Экономисты объяснили, почему продавцам на аукционе выгодно держать свою цену в тайне'

Достанем свойства этой ссылки (куда ведет, class)

In [49]:
attrs = title_obj.attrs
attrs

{'href': '/news/science/530853899.html',
 'class': ['link', 'link_dark2', 'no-visited']}

Достанем саму ссылку

In [50]:
href = title_obj.attrs['href']
href

'/news/science/530853899.html'

Достанем текст новости

In [51]:
short_text = news[0].find('div', {'class': 'post__text'}).text
short_text

'Экономические модели прошлых лет показывают, что организаторам аукционов стоит открыто называть цену, ниже которой они не согласятся продать свой товар. Однако, как доказали исследователи из ВШЭ и Университета Торонто, в некоторых случаях оптимальной является обратная стратегия. Результаты\xa0работы\xa0были\xa0представлены\xa0на семинаре департамента теоретической экономики ВШЭ.'

Достанем день, месяц, год публикации

In [52]:
pub_day = news[0].find('div', {'class': 'post-meta__day'}).text
pub_day

'18'

In [53]:
pub_month = news[0].find('div', {'class': 'post-meta__month'}).text
pub_month

'ноя'

In [54]:
pub_year = news[0].find('div', {'class': 'post-meta__year'}).text
pub_year

'2021'

**Шаг 2. Научиться парсить страничку самой новости**

Возьмем ссылку на полную новость и соединим с адерсом сайта (т.к. ссылка относительная)

In [55]:
url_one = 'http://www.hse.ru' + href
url_one

'http://www.hse.ru/news/science/530853899.html'

Скачаем ее и распарсим

In [56]:
req = session.get(url_one, headers={'User-Agent': ua.random})
page = req.text

soup = BeautifulSoup(page, 'html.parser')

Сохраним текст, распечатаем кусочек

In [57]:
full_text = soup.find('div', {'class': 'post__content'}).text
full_text[:200]

'Экономисты объяснили, почему продавцам на аукционе выгодно держать свою цену в тайне© iStockЭкономические модели прошлых лет показывают, что организаторам аукционов стоит открыто называть цену, ниже к'

Найдем теги, которые присвоены статье

In [58]:
meta = soup.find('div', {'class': 'articleMeta'})

tags = meta.find_all('a', {'class': 'tag'})
tags = [t.text for t in tags]
tags

['исследования и аналитика', 'взгляд ученого', 'международное сотрудничество']

**Шаг 3. Оформляем нормально в функции**

Сделаем словарь соответствий имени месяца и его номера

In [59]:
months = {
    value: key + 1
    for key, value in enumerate(
        ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    )
}

Парсим информацию из страницы со списком новостей (блок одной новости)

In [64]:
def parse_news_page_block(one_block):
    block = {}
    a = one_block.find('a')
    block['title'] = a.text
    block['href'] = a.attrs['href']
    block['short_text'] = one_block.find('div', {'class': 'post__text'}).text
    block['pub_day'] = int(one_block.find('div', {'class': 'post-meta__day'}).text)
    block['pub_month'] = months[one_block.find('div', {'class': 'post-meta__month'}).text]
    block['pub_year'] = int(one_block.find('div', {'class': 'post-meta__year'}).text)
    return block

Парсим отдельную страницу новости

In [61]:
def parse_one_article(block):
    url_one = 'http://www.hse.ru' + block['href']
    req = session.get(url_one, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    block['full_text'] = soup.find('div', {'class': 'post__content'}).text
    meta = soup.find('div', {'class': 'articleMeta'})
    tags = meta.find_all('a', {'class': 'tag'})
    block['tags'] = [t.text for t in tags]     
    return block

Регулярное выражение для того, чтобы достать ID новости и не повторяться

In [62]:
regex_hse_id = re.compile('/([0-9]*?).html')

Обработать N-ую страницу новостей

In [63]:
def get_nth_page(page_number):
    # скачиваем
    url = f'https://www.hse.ru/news/page{page_number}.html'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    
    # находим новости
    news = soup.find_all('div', {'class': 'post'})
    
    # идем по новостям и обрабатываем их
    blocks = []
    for n in news:
        try:
            blocks.append(parse_news_page_block(n))
        except Exception as e:
            print(e)

    # идем по отдельным статьям и достаем информацию
    result = []
    for b in blocks:
        if b['href'].startswith('/'):
            idx = regex_hse_id.findall(b['href'])[0]
            if idx not in seen_news:
                try:
                    res = parse_one_article(b)
                    res['hse_id'] = idx
                    result.append(res)
                except Exception as e:
                    print(e)
            else:
                print('Seen', b['href'])
    
    # возвращаем найденную информацию
    return result

**Шаг 4. Пишем в базу**

Надо завести словарь для тегов (сначала читаем из базы, а потом дозаписываем), множество виденных статей (чтобы при перезаупске не дублировать)

In [93]:
def write_to_db(block):
    
    # обрабатываем теги
    tags = []
    for tag in block['tags']:
        # если тег новый для нас, то присваиваем ему ID и сохраняем в базу
        if tag not in db_tags:
            if db_tags.values():
                db_tags[tag] = max(db_tags.values()) + 1 
            else:
                db_tags[tag] = 1
            cur.execute('INSERT INTO tags VALUES (?, ?)', (len(db_tags), tag))
            conn.commit()
        # добавляем номер в список будущих соответствий
        tags.append(db_tags[tag])
    
    # сохраняем информацию по текстам
    cur.execute(
        """
        INSERT INTO texts 
            (hse_id, pub_year, pub_month, pub_day, title, short_text, full_text) 
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            block['hse_id'],
            block['pub_year'], block['pub_month'], block['pub_day'],
            block['title'], block['short_text'], block['full_text'])
    )
    
    # достаем id текста
    cur.execute("SELECT id FROM texts WHERE hse_id = ?", (block['hse_id'],))
    text_id = cur.fetchone()[0]
    
    # содаем список соответствий тегов
    tags = [(text_id, t) for t in tags]
    
    # сохраняем в таблицу
    cur.executemany(
        'INSERT INTO text_to_tag (id_text, id_tag) VALUES (?, ?)',
        tags
    )
    
    conn.commit()
    
    # добавляем, что такой id уже видели
    seen_news.add(block['hse_id'])

In [99]:
conn = sqlite3.connect('hse_news.db')
cur = conn.cursor()

In [100]:
# создаем словарь тегов и их идентификаторов
cur.execute('SELECT tag_name, id FROM tags')
db_tags = {}
for name, idx in cur.fetchall():
    db_tags[name] = idx

# достаем, какие новости уже видели (если потом захотим обновлять базу)
cur.execute('SELECT hse_id FROM texts')
seen_news = set(i[0] for i in cur.fetchall())

In [101]:
from tqdm.auto import tqdm

Напишем функцию, куда передаем количество страниц и она выполняет все нужные действия

In [102]:
def run_all(n_pages):
    for i in tqdm(range(n_pages)):
        blocks = get_nth_page(i+1)
        for block in blocks:
            write_to_db(block)

Запускаем на 20 первых страниц

In [103]:
run_all(20)

  0%|          | 0/20 [00:00<?, ?it/s]

Посмотрим на 10 самых популярных тегов

In [104]:
cur.execute("""
SELECT count(text_to_tag.id) as cnt, tags.tag_name 
    FROM text_to_tag 
        JOIN tags ON tags.id = text_to_tag.id_tag 
            GROUP BY text_to_tag.id_tag 
            ORDER BY cnt DESC
            LIMIT 10;
""")
cur.fetchall()

[(61, 'репортаж о событии'),
 (43, 'исследования и аналитика'),
 (41, 'идеи и опыт'),
 (34, 'дискуссии'),
 (32, 'взгляд ученого'),
 (31, 'достижения'),
 (27, 'студенты'),
 (27, 'приглашение к участию'),
 (21, 'новое в ВШЭ'),
 (17, 'профессора')]

Посмотрим, сколько публикаций по месяцам

In [105]:
cur.execute("""
SELECT count(pub_month) as cnt, pub_month
    FROM texts
        GROUP BY pub_month
        ORDER BY cnt DESC;
""")
cur.fetchall()

[(83, 10), (43, 9), (40, 11)]